In [1]:
import pandas as pd
import geopandas as gpd

In [4]:
all_NUTS_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\nuts3_2003_geom_10.gpkg")

In [6]:
all_NUTS_coord[all_NUTS_coord["geocode"]=="ES612"]

,geocode,etiqueta,granularidad,gcd_nuts0,gcd_nuts1,gcd_nuts2,longitud,latitud,geometry
646,ES612,Cádiz,NUTS3,ES,ES6,ES61,-5.7648,36.5263,"MULTIPOLYGON (((-5.25241 36.31128, -5.25677 36..."


In [8]:
airport_coord=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.8\infrastructure\airports_info\airports_coordinates_v1.1.csv")

In [11]:
airport_coord

,icao_id,lat,lon
0,AGGA,-8.698333,160.678333
1,AGGE,-7.083333,155.750000
2,AGGH,-9.431667,160.053333
3,AGGJ,-9.533333,160.083333
4,AGGK,-10.450000,161.896667
...,...,...,...
9958,GOBD,14.670960,-17.072830
9959,LECH,40.213890,0.073330
9960,UGGG,41.668000,44.952500
9961,LTFM,41.261300,28.741950


In [12]:
airport_coord.dtypes

icao_id     object
lat        float64
lon        float64
dtype: object

In [14]:
andalusia=all_NUTS_coord[all_NUTS_coord["geocode"].str.contains("ES61")]

In [16]:
andalusia

,geocode,etiqueta,granularidad,gcd_nuts0,gcd_nuts1,gcd_nuts2,longitud,latitud,geometry
645,ES611,Almería,NUTS3,ES,ES6,ES61,-2.3851,37.2985,"MULTIPOLYGON (((-3.12869 36.75089, -3.13286 36..."
646,ES612,Cádiz,NUTS3,ES,ES6,ES61,-5.7648,36.5263,"MULTIPOLYGON (((-5.25241 36.31128, -5.25677 36..."
647,ES613,Córdoba,NUTS3,ES,ES6,ES61,-4.7934,37.9567,"MULTIPOLYGON (((-5.58485 38.13175, -5.58586 38..."
648,ES614,Granada,NUTS3,ES,ES6,ES61,-3.2676,37.3887,"MULTIPOLYGON (((-2.20763 37.91659, -2.25074 37..."
649,ES615,Huelva,NUTS3,ES,ES6,ES61,-6.8223,37.5017,"MULTIPOLYGON (((-6.33107 36.89884, -6.33989 36..."
650,ES616,Jaén,NUTS3,ES,ES6,ES61,-3.3615,37.9556,"MULTIPOLYGON (((-4.00084 37.40211, -4.03105 37..."
651,ES617,Málaga,NUTS3,ES,ES6,ES61,-4.6890,36.7964,"MULTIPOLYGON (((-5.25241 36.31128, -5.25954 36..."
652,ES618,Sevilla,NUTS3,ES,ES6,ES61,-5.5960,37.5198,"MULTIPOLYGON (((-6.33107 36.89884, -6.36388 36..."


In [36]:
gdf_airports=gpd.GeoDataFrame(airport_coord,geometry=gpd.points_from_xy(airport_coord["lon"],airport_coord["lat"]),crs="EPSG:4326")

In [37]:
gdf_airports.columns

Index(['icao_id', 'lat', 'lon', 'geometry'], dtype='object')

In [25]:
polygon=andalusia["geometry"].iloc[0]

In [50]:
def airports_in_nuts(airports,nuts):
    airports_within_polygon=pd.DataFrame(columns=['icao_id', 'lat', 'lon', 'geometry'])
    for _, row in nuts.iterrows():
        polygon=row["geometry"]
        airports_in_this_polygon = airports[airports.within(polygon)]
        pd.concat([airports_within_polygon, airports_in_this_polygon])
    return airports_within_polygon


In [33]:
gdf_airports

,icao_id,lat,lon,geometry
0,AGGA,-8.698333,160.678333,POINT (160.67833 -8.69833)
1,AGGE,-7.083333,155.750000,POINT (155.75 -7.08333)
2,AGGH,-9.431667,160.053333,POINT (160.05333 -9.43167)
3,AGGJ,-9.533333,160.083333,POINT (160.08333 -9.53333)
4,AGGK,-10.450000,161.896667,POINT (161.89667 -10.45)
...,...,...,...,...
9958,GOBD,14.670960,-17.072830,POINT (-17.07283 14.67096)
9959,LECH,40.213890,0.073330,POINT (0.07333 40.21389)
9960,UGGG,41.668000,44.952500,POINT (44.9525 41.668)
9961,LTFM,41.261300,28.741950,POINT (28.74195 41.2613)


In [53]:
def airports_in_nuts(airports, nuts):
    # Create an empty list to store results
    results = []

    for _, row in nuts.iterrows():
        polygon = row["geometry"]  # Get the polygon

        # Select airports within the polygon
        airports_in_this_polygon = airports[airports["geometry"].within(polygon)]

        # Append only if there are matching airports
        if not airports_in_this_polygon.empty:
            results.append(airports_in_this_polygon)

    # Concatenate only if results exist
    if results:
        airports_within_polygon = pd.concat(results, ignore_index=True)
    else:
        airports_within_polygon = pd.DataFrame(columns=airports.columns)

    return airports_within_polygon

In [54]:
result=airports_in_nuts(gdf_airports,andalusia)

In [55]:
result

,icao_id,lat,lon,geometry
0,LEAM,36.843889,-2.370000,POINT (-2.37 36.84389)
1,LEFA,36.648333,-6.347783,POINT (-6.34778 36.64833)
2,LEJR,36.744722,-6.060000,POINT (-6.06 36.74472)
3,LERT,36.645000,-6.349444,POINT (-6.34944 36.645)
4,LESF,36.484444,-6.185278,POINT (-6.18528 36.48444)
5,LETJ,36.861389,-6.140556,POINT (-6.14056 36.86139)
6,LEBA,37.841944,-4.848889,POINT (-4.84889 37.84194)
7,LEPR,37.716944,-5.212500,POINT (-5.2125 37.71694)
8,LEGA,37.133333,-3.635833,POINT (-3.63583 37.13333)
9,LEGR,37.188611,-3.777222,POINT (-3.77722 37.18861)


In [1]:
new_kpas= {"efficiency","interoperability","flexibility","cost effectiveness","capacity","environment"}

In [2]:
old_kpas={"capacity","efficiency","accessibility","flexibility","environment","resilience"}

In [3]:
new_kpas-old_kpas

{'cost effectiveness', 'interoperability'}

In [4]:
old_kpas-new_kpas

{'accessibility', 'resilience'}